# Download fiji and deepimagej

In [ ]:
!mkdir /home/jovyan/
!wget https://downloads.imagej.net/fiji/latest/fiji-linux64.zip
!unzip fiji-linux64.zip -d /home/jovyan
!wget https://github.com/deepimagej/deepimagej-plugin/releases/download/2.1.7-SNAPSHOT/DeepImageJ_-2.1.7-SNAPSHOT.jar
!mv DeepImageJ_-2.1.7-SNAPSHOT.jar /home/jovyan/Fiji.app/plugins
!wget https://github.com/deepimagej/deepimagej-plugin/releases/download/2.1.7-SNAPSHOT/dependencies_217.zip
!unzip dependencies_217.zip
!mv -f dependencies_2.1.7/* /home/jovyan/Fiji.app/jars/

# ImageJ with Python Kernel

This notebook covers how to use ImageJ as a library from Python. A major advantage of this approach is the ability to combine ImageJ with other tools available from the Python software ecosystem, including [NumPy](https://www.numpy.org/), [SciPy](https://www.scipy.org/), [scikit-image](https://scikit-image.org/), [CellProfiler](https://cellprofiler.org/), [OpenCV](https://opencv.org/), [ITK](https://itk.org/) and more.

This notebook assumes familiarity with the ImageJ API. Detailed tutorials in that regard can be found in the [other notebooks](../ImageJ-Tutorials-and-Demo.ipynb).

## Starting ImageJ from Python

The [pyimagej](https://pypi.org/project/pyimagej/) module enables access to the entire ImageJ API from Python in a natural way.

Let's initialize an ImageJ gateway including Fiji plugins, at a reproducible version:

In [ ]:
# fix starting fiji issue https://forum.image.sc/t/fiji-fails-to-launch-after-update/44582
!rm /home/jovyan/Fiji.app/jars/FilamentDetector-1.0.0.jar

In [ ]:
import imagej
ij = imagej.init('/home/jovyan/Fiji.app',headless=False)
ij.getVersion()

# Download a deepimagej model

In [ ]:
!wget  https://zenodo.org/record/4608442/files/SMLM_Density%20Map_Estimation_%28DEFCoN%29.bioimage.io.model.zip
!rm -rf /home/jovyan/Fiji.app/models/DEFCoN.bioimage.io.model
!mkdir -p /home/jovyan/Fiji.app/models/DEFCoN.bioimage.io.model
!unzip "SMLM_Density Map_Estimation_(DEFCoN).bioimage.io.model.zip" -d /home/jovyan/Fiji.app/models/DEFCoN.bioimage.io.model/

In [ ]:
!ls /home/jovyan/Fiji.app/models/DEFCoN.bioimage.io.model/

In [ ]:
macro = """
#@ String inputFile
#@output String logString

open("/home/jovyan/Fiji.app/models/DEFCoN.bioimage.io.model/exampleImage.tif");
selectWindow("exampleImage.tif");
saveAs("PNG", "/home/jovyan/input.png");
run("DeepImageJ Run", "model=[SMLM Density Map Estimation (DEFCoN)] format=Tensorflow preprocessing=[preprocessing.txt] postprocessing=[postprocessing.txt] axes=Y,X,C tile=84,84,1 logging=normal");
selectWindow("DEFCoN density map estimation of exampleImage.tif");
saveAs("PNG", "/home/jovyan/output.png");

IJ.log("Done");
logString = getInfo("log");
"""
args = {
    'inputFile': '"/home/jovyan/Fiji.app/models/DEFCoN.bioimage.io.model/exampleImage.tif"',
}
result = ij.py.run_macro(macro, args)
print(result.getOutput('logString')) # it should print '...Done'

In [ ]:
from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')
# ij.py.run_macro("""run("Blobs (25K)");""")
blobs = WindowManager.getCurrentImage()
print(blobs)
ij.py.show(blobs)